In [15]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [16]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [17]:
subject = 'Santa Catarina - Consumo de Cimento (t)'
split_index = 191 #Referente aos 230 anos de input  
train_split = split_index + 1 -30

In [18]:
data = pd.read_csv('2003_mo_model_input_SC.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Santa Catarina - IDH,Santa Catarina - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Santa Catarina - Desemprego,Santa Catarina - Produção de Cimento (t),Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado,Santa Catarina - Consumo de Cimento (t)
0,2003-1,0.797768,0.311113,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,8.256606,20.216800,1.320065e+08,6.638363e+06,19.725168,1.221860e+08,142.456
1,2003-2,0.797853,0.312787,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,8.250731,19.631031,1.321743e+08,6.642080e+06,19.732277,1.222513e+08,135.829
2,2003-3,0.797939,0.314207,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,8.244856,19.400920,1.323421e+08,6.645797e+06,19.739386,1.223166e+08,133.135
3,2003-4,0.798024,0.315571,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,8.238982,19.153741,1.325098e+08,6.649513e+06,19.746495,1.223818e+08,144.723
4,2003-5,0.798110,0.316880,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,8.233107,18.912155,1.326776e+08,6.653230e+06,19.753604,1.224471e+08,150.523
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,0.663849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,155.321494,NaN,NaN,NaN,NaN,377.545
236,2022-9,NaN,0.663153,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,154.790929,NaN,NaN,NaN,NaN,370.282
237,2022-10,NaN,0.661797,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,155.428377,NaN,NaN,NaN,NaN,340.900
238,2022-11,NaN,0.660090,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,155.448086,NaN,NaN,NaN,NaN,363.900


In [19]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Santa Catarina - IDH,Santa Catarina - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Santa Catarina - Desemprego,Santa Catarina - Produção de Cimento (t),Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado
0,-2.119998,-1.030889,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,1.085359,-1.154511,-1.682734,-2.417084,-3.022828,-2.152611
1,-2.088248,-1.010380,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,1.080851,-1.166496,-1.664640,-2.359361,-2.949421,-2.116149
2,-2.056499,-0.992988,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,1.076343,-1.171204,-1.646546,-2.301638,-2.876014,-2.079688
3,-2.024750,-0.976287,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,1.071835,-1.176262,-1.628452,-2.243915,-2.802607,-2.043226
4,-1.993000,-0.960251,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,1.067328,-1.181205,-1.610358,-2.186191,-2.729200,-2.006765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.726097,1.713104,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,-1.126590,0.933479,1.196588,-0.578473,0.290543,1.029069
188,0.689854,1.722266,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,-1.131368,0.953950,1.189460,-0.568205,0.281308,1.023160
189,0.653611,1.738852,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,-1.136147,0.982622,1.182332,-0.557938,0.272072,1.017252
190,0.617368,1.755526,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,-1.140925,1.015842,1.175204,-0.547670,0.262837,1.011344


In [20]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      135.344
1      127.693
2      151.570
3      137.044
4      134.493
        ...   
235        NaN
236        NaN
237        NaN
238        NaN
239        NaN
Name: Santa Catarina - Consumo de Cimento (t), Length: 240, dtype: float64

In [21]:
# input para treinamento
train_input = input_data.iloc[:train_split]
train_input

,Santa Catarina - IDH,Santa Catarina - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Santa Catarina - Desemprego,Santa Catarina - Produção de Cimento (t),Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado
0,-2.119998,-1.030889,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,1.085359,-1.154511,-1.682734,-2.417084,-3.022828,-2.152611
1,-2.088248,-1.010380,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,1.080851,-1.166496,-1.664640,-2.359361,-2.949421,-2.116149
2,-2.056499,-0.992988,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,1.076343,-1.171204,-1.646546,-2.301638,-2.876014,-2.079688
3,-2.024750,-0.976287,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,1.071835,-1.176262,-1.628452,-2.243915,-2.802607,-2.043226
4,-1.993000,-0.960251,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,1.067328,-1.181205,-1.610358,-2.186191,-2.729200,-2.006765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,1.514485,1.376970,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,-0.817925,0.890411,1.181211,-0.551568,0.108581,0.958937
158,1.496494,1.401965,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,-0.835333,0.883759,1.187451,-0.577579,0.126542,0.968867
159,1.478503,1.424150,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,-0.852741,0.877272,1.193692,-0.603589,0.144504,0.978796
160,1.460512,1.443460,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,-0.870149,0.861580,1.199932,-0.629600,0.162465,0.988726


In [22]:
# Alvo para treinamento
train_target = target_data.iloc[:train_split]
train_target

0      135.344
1      127.693
2      151.570
3      137.044
4      134.493
        ...   
157    211.861
158    258.760
159    211.393
160    249.531
161    231.359
Name: Santa Catarina - Consumo de Cimento (t), Length: 162, dtype: float64

In [23]:
#input de test (Ano 2021)
test_input = input_data.iloc[train_split:]
test_input

,Santa Catarina - IDH,Santa Catarina - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Santa Catarina - Desemprego,Santa Catarina - Produção de Cimento (t),Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado
162,1.424530,1.485895,-0.601510,-0.653307,0.157879,-1.255326,1.088045,0.763466,-1.213929,-0.904966,0.863324,1.212413,-0.681621,0.198388,1.008586
163,1.406539,1.509096,-0.786068,-0.653814,0.548855,-1.118679,1.099377,0.752299,-1.292173,-0.922374,0.865440,1.218653,-0.707632,0.216350,1.018515
164,1.388548,1.521923,-0.830387,-0.645656,0.385685,-0.936336,1.110709,0.741131,-1.324219,-0.939782,0.861470,1.224893,-0.733643,0.234311,1.028445
165,1.370557,1.534157,-0.801089,-0.634274,0.420965,-0.931790,1.122042,0.729964,-1.344446,-0.957191,0.856540,1.231133,-0.759653,0.252273,1.038375
166,1.352566,1.545781,-0.959917,-0.624035,0.548870,-1.168522,1.133374,0.718796,-1.381638,-0.974599,0.858615,1.237374,-0.785664,0.270234,1.048305
167,1.334575,1.556827,-1.022309,-0.619791,0.222499,-1.285611,1.144706,0.707629,-1.411208,-0.992007,0.859134,1.243614,-0.811674,0.288196,1.058235
168,1.316584,1.564835,-1.074401,-0.620112,-0.614754,-1.446316,1.156038,0.696461,-1.412953,-1.009415,0.862668,1.249854,-0.837685,0.306157,1.068164
169,1.288519,1.572190,-1.119597,-0.615973,-0.478229,-1.357832,1.167287,0.681823,-1.491464,-1.016393,0.869114,1.249573,-0.822073,0.310243,1.068353
170,1.260453,1.581429,-1.078648,-0.609777,-0.393157,-1.235682,1.178536,0.667184,-1.573805,-1.023370,0.871734,1.249293,-0.806462,0.314330,1.068541
171,1.232388,1.590193,-1.055426,-0.602141,-0.557552,-1.065340,1.189784,0.652545,-1.564950,-1.030347,0.874469,1.249012,-0.790850,0.318416,1.068730


In [24]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[train_split:split_index + 1]
test_target

162    265.999
163    277.169
164    245.824
165    244.227
166    263.751
167    211.136
168    219.002
169    234.275
170    249.920
171    246.326
172    198.778
173    285.970
174    262.597
175    277.811
176    235.885
177    272.671
178    278.592
179    214.658
180    252.700
181    246.146
182    255.873
183    273.299
184    270.918
185    256.605
186    303.526
187    299.284
188    284.550
189    307.576
190    281.317
191    226.233
Name: Santa Catarina - Consumo de Cimento (t), dtype: float64

In [25]:
def validation_splitter(df, div_factor):
    split_factor = len(df)//div_factor
    positions_to_drop = []
    for i in range(split_factor):
        pos = df.shape[0] - (i*div_factor + 1)
        positions_to_drop.append(pos)
    
    return df.drop(positions_to_drop), df.iloc[positions_to_drop]

In [26]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train)
#     display(train_val)
#     display(target)
#     display(target_val)
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val, target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [27]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(25)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [28]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[1609715626, 4128121810, 1732235509, 1097717160, 2840762640, 2953024162, 3018457859, 467238027, 2858542459, 2594737874, 1393387487, 3766779401, 2019487258, 3831108568, 2064519234, 612046021, 3972114233, 3735857573, 1549041136, 2431116659, 4140783039, 1512828274, 240834437, 312163355, 1382641550]


Step: 0 ___________________________________________
val_loss: 292.4897766113281
winner_seed: 1609715626


Step: 1 ___________________________________________
val_loss: 328.99212646484375


Step: 2 ___________________________________________
val_loss: 306.8448181152344


Step: 3 ___________________________________________
val_loss: 260.78070068359375
winner_seed: 1097717160


Step: 4 ___________________________________________
val_loss: 293.7254638671875


Step: 5 ___________________________________________
val_loss: 302.55841064453125


Step: 6 ___________________________________________
val_loss: 296.88311767578125


Step: 7 ___________________________________________
val_loss: 305.618438720

In [29]:
trained_model, history = neural_network_model(train_input, 
                                              train_target, 
                                              want_verbose=1, 
                                              seed=winner_seed)

Epoch 1/10000
5/5 [==============================] - 1s 24ms/step - loss: 57288.8867 - val_loss: 62680.2109
Epoch 2/10000
5/5 [==============================] - 0s 5ms/step - loss: 40512.1602 - val_loss: 39243.5234
Epoch 3/10000
5/5 [==============================] - 0s 5ms/step - loss: 38857.0742 - val_loss: 16308.1309
Epoch 4/10000
5/5 [==============================] - 0s 5ms/step - loss: 15704.0059 - val_loss: 11719.1084
Epoch 5/10000
5/5 [==============================] - 0s 6ms/step - loss: 19617.2051 - val_loss: 27996.5371
Epoch 6/10000
5/5 [==============================] - 0s 6ms/step - loss: 10637.6455 - val_loss: 2075.9858
Epoch 7/10000
5/5 [==============================] - 0s 6ms/step - loss: 1098.8662 - val_loss: 700.0132
Epoch 8/10000
5/5 [==============================] - 0s 5ms/step - loss: 657.9298 - val_loss: 1440.9015
Epoch 9/10000
5/5 [==============================] - 0s 6ms/step - loss: 813.8293 - val_loss: 669.4579
Epoch 10/10000
5/5 [===========================

5/5 [==============================] - 0s 5ms/step - loss: 328.0044 - val_loss: 410.6747
Epoch 80/10000
5/5 [==============================] - 0s 5ms/step - loss: 339.7581 - val_loss: 420.8201
Epoch 81/10000
5/5 [==============================] - 0s 6ms/step - loss: 342.6650 - val_loss: 495.6264
Epoch 82/10000
5/5 [==============================] - 0s 5ms/step - loss: 435.4477 - val_loss: 638.8975
Epoch 83/10000
5/5 [==============================] - 0s 5ms/step - loss: 405.2791 - val_loss: 547.3588
Epoch 84/10000
5/5 [==============================] - 0s 6ms/step - loss: 352.3643 - val_loss: 455.9836
Epoch 85/10000
5/5 [==============================] - 0s 5ms/step - loss: 362.9444 - val_loss: 486.0700
Epoch 86/10000
5/5 [==============================] - 0s 5ms/step - loss: 376.1654 - val_loss: 433.0258
Epoch 87/10000
5/5 [==============================] - 0s 5ms/step - loss: 347.8734 - val_loss: 450.7546
Epoch 88/10000
5/5 [==============================] - 0s 5ms/step - loss: 332.3

5/5 [==============================] - 0s 5ms/step - loss: 349.2169 - val_loss: 473.3010
Epoch 158/10000
5/5 [==============================] - 0s 6ms/step - loss: 344.9694 - val_loss: 355.5154
Epoch 159/10000
5/5 [==============================] - 0s 6ms/step - loss: 336.9128 - val_loss: 427.4331
Epoch 160/10000
5/5 [==============================] - 0s 5ms/step - loss: 323.4824 - val_loss: 453.7493
Epoch 161/10000
5/5 [==============================] - 0s 5ms/step - loss: 498.5740 - val_loss: 1057.3849
Epoch 162/10000
5/5 [==============================] - 0s 5ms/step - loss: 3775.9871 - val_loss: 4164.5620
Epoch 163/10000
5/5 [==============================] - 0s 5ms/step - loss: 13643.7773 - val_loss: 10604.1260
Epoch 164/10000
5/5 [==============================] - 0s 5ms/step - loss: 12692.0117 - val_loss: 4825.2656
Epoch 165/10000
5/5 [==============================] - 0s 5ms/step - loss: 3752.0889 - val_loss: 584.6418
Epoch 166/10000
5/5 [==============================] - 0s 5m

5/5 [==============================] - 0s 5ms/step - loss: 312.4682 - val_loss: 448.1434
Epoch 236/10000
5/5 [==============================] - 0s 5ms/step - loss: 378.8308 - val_loss: 408.2769
Epoch 237/10000
5/5 [==============================] - 0s 6ms/step - loss: 373.5367 - val_loss: 450.0397
Epoch 238/10000
5/5 [==============================] - 0s 6ms/step - loss: 399.3152 - val_loss: 451.1255
Epoch 239/10000
5/5 [==============================] - 0s 7ms/step - loss: 332.9599 - val_loss: 401.2848
Epoch 240/10000
5/5 [==============================] - 0s 6ms/step - loss: 323.5499 - val_loss: 421.9679
Epoch 241/10000
5/5 [==============================] - 0s 6ms/step - loss: 336.9258 - val_loss: 392.0228
Epoch 242/10000
5/5 [==============================] - 0s 5ms/step - loss: 394.8406 - val_loss: 379.5114
Epoch 243/10000
5/5 [==============================] - 0s 5ms/step - loss: 331.8983 - val_loss: 365.3628
Epoch 244/10000
5/5 [==============================] - 0s 5ms/step - lo

5/5 [==============================] - 0s 5ms/step - loss: 391.8003 - val_loss: 498.4064
Epoch 314/10000
5/5 [==============================] - 0s 5ms/step - loss: 330.9959 - val_loss: 484.3545
Epoch 315/10000
5/5 [==============================] - 0s 5ms/step - loss: 326.7340 - val_loss: 525.2438
Epoch 316/10000
5/5 [==============================] - 0s 6ms/step - loss: 370.4625 - val_loss: 534.3295
Epoch 317/10000
5/5 [==============================] - 0s 6ms/step - loss: 434.0988 - val_loss: 588.6291
Epoch 318/10000
5/5 [==============================] - 0s 5ms/step - loss: 417.2661 - val_loss: 645.8949
Epoch 319/10000
5/5 [==============================] - 0s 5ms/step - loss: 438.5542 - val_loss: 478.2332
Epoch 320/10000
5/5 [==============================] - 0s 5ms/step - loss: 379.4451 - val_loss: 630.0348
Epoch 321/10000
5/5 [==============================] - 0s 5ms/step - loss: 473.2534 - val_loss: 634.4398
Epoch 322/10000
5/5 [==============================] - 0s 5ms/step - lo

...

5/5 [==============================] - 0s 6ms/step - loss: 334.4478 - val_loss: 438.8349
Epoch 782/10000
5/5 [==============================] - 0s 5ms/step - loss: 281.1400 - val_loss: 399.1999
Epoch 783/10000
5/5 [==============================] - 0s 5ms/step - loss: 277.7169 - val_loss: 360.7577
Epoch 784/10000
5/5 [==============================] - 0s 5ms/step - loss: 268.7731 - val_loss: 345.2165
Epoch 785/10000
5/5 [==============================] - 0s 5ms/step - loss: 267.1143 - val_loss: 418.9637
Epoch 786/10000
5/5 [==============================] - 0s 5ms/step - loss: 282.8325 - val_loss: 432.6539
Epoch 787/10000
5/5 [==============================] - 0s 7ms/step - loss: 289.8036 - val_loss: 437.0727
Epoch 788/10000
5/5 [==============================] - 0s 6ms/step - loss: 287.1727 - val_loss: 396.0342
Epoch 789/10000
5/5 [==============================] - 0s 6ms/step - loss: 260.9545 - val_loss: 435.9117
Epoch 790/10000
5/5 [==============================] - 0s 5ms/step - lo

5/5 [==============================] - 0s 6ms/step - loss: 309.9396 - val_loss: 450.1852
Epoch 860/10000
5/5 [==============================] - 0s 5ms/step - loss: 326.5946 - val_loss: 387.4388
Epoch 861/10000
5/5 [==============================] - 0s 6ms/step - loss: 308.3000 - val_loss: 340.5284
Epoch 862/10000
5/5 [==============================] - 0s 5ms/step - loss: 273.9050 - val_loss: 332.1570
Epoch 863/10000
5/5 [==============================] - 0s 5ms/step - loss: 290.8457 - val_loss: 336.8918
Epoch 864/10000
5/5 [==============================] - 0s 5ms/step - loss: 299.1246 - val_loss: 398.9407
Epoch 865/10000
5/5 [==============================] - 0s 5ms/step - loss: 295.9574 - val_loss: 429.0801
Epoch 866/10000
5/5 [==============================] - 0s 5ms/step - loss: 323.8509 - val_loss: 459.8977
Epoch 867/10000
5/5 [==============================] - 0s 6ms/step - loss: 310.2760 - val_loss: 409.8208
Epoch 868/10000
5/5 [==============================] - 0s 7ms/step - lo

5/5 [==============================] - 0s 5ms/step - loss: 288.5234 - val_loss: 405.3542
Epoch 938/10000
5/5 [==============================] - 0s 5ms/step - loss: 272.4277 - val_loss: 415.8971
Epoch 939/10000
5/5 [==============================] - 0s 5ms/step - loss: 315.0876 - val_loss: 417.8173
Epoch 940/10000
5/5 [==============================] - 0s 5ms/step - loss: 269.1986 - val_loss: 373.2932
Epoch 941/10000
5/5 [==============================] - 0s 5ms/step - loss: 268.6543 - val_loss: 389.6290
Epoch 942/10000
5/5 [==============================] - 0s 6ms/step - loss: 267.4656 - val_loss: 428.0318
Epoch 943/10000
5/5 [==============================] - 0s 5ms/step - loss: 266.4953 - val_loss: 469.7852
Epoch 944/10000
5/5 [==============================] - 0s 5ms/step - loss: 280.3837 - val_loss: 380.5300
Epoch 945/10000
5/5 [==============================] - 0s 6ms/step - loss: 309.9528 - val_loss: 350.2215
Epoch 946/10000
5/5 [==============================] - 0s 5ms/step - lo

5/5 [==============================] - 0s 6ms/step - loss: 283.2842 - val_loss: 396.8042
Epoch 1016/10000
5/5 [==============================] - 0s 5ms/step - loss: 297.1708 - val_loss: 413.3251
Epoch 1017/10000
5/5 [==============================] - 0s 6ms/step - loss: 329.8954 - val_loss: 416.8189
Epoch 1018/10000
5/5 [==============================] - 0s 6ms/step - loss: 286.0403 - val_loss: 427.0500
Epoch 1019/10000
5/5 [==============================] - 0s 5ms/step - loss: 399.7403 - val_loss: 387.7244
Epoch 1020/10000
5/5 [==============================] - 0s 7ms/step - loss: 338.2754 - val_loss: 429.3354
Epoch 1021/10000
5/5 [==============================] - 0s 6ms/step - loss: 355.3056 - val_loss: 422.7171
Epoch 1022/10000
5/5 [==============================] - 0s 6ms/step - loss: 371.3227 - val_loss: 492.6704
Epoch 1023/10000
5/5 [==============================] - 0s 6ms/step - loss: 375.9057 - val_loss: 469.6330
Epoch 1024/10000
5/5 [==============================] - 0s 5ms/

5/5 [==============================] - 0s 5ms/step - loss: 336.0671 - val_loss: 389.0568
Epoch 1093/10000
5/5 [==============================] - 0s 5ms/step - loss: 310.0309 - val_loss: 402.8286
Epoch 1094/10000
5/5 [==============================] - 0s 5ms/step - loss: 301.5499 - val_loss: 413.5443
Epoch 1095/10000
5/5 [==============================] - 0s 5ms/step - loss: 289.5688 - val_loss: 381.5125
Epoch 1096/10000
5/5 [==============================] - 0s 5ms/step - loss: 309.3739 - val_loss: 411.7959
Epoch 1097/10000
5/5 [==============================] - 0s 6ms/step - loss: 334.3186 - val_loss: 447.9141
Epoch 1098/10000
5/5 [==============================] - 0s 6ms/step - loss: 328.0647 - val_loss: 436.5893
Epoch 1099/10000
5/5 [==============================] - 0s 5ms/step - loss: 309.8727 - val_loss: 364.0591
Epoch 1100/10000
5/5 [==============================] - 0s 5ms/step - loss: 287.3180 - val_loss: 440.0021
Epoch 1101/10000
5/5 [==============================] - 0s 5ms/

In [30]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input.iloc[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [31]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            test_input, 
                                            test_target, 
                                            train_split)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 14ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,226.551666,226.449753,226.449722,226.449661,226.449448,226.449371,226.449783,226.449448,226.449265,226.444046,226.447556,225.839218,225.921066,233.727737,225.763092,251.292374,246.586472,248.284378,245.010376,245.91713,232.542099,223.657089,155.955246,101.290787,20.778214,-37.767906,-38.867241,-38.096649,-37.516937,4.947762
Target,265.999,277.169,245.824,244.227,263.751,211.136,219.002,234.275,249.92,246.326,198.778,285.97,262.597,277.811,235.885,272.671,278.592,214.658,252.7,246.146,255.873,273.299,270.918,256.605,303.526,299.284,284.55,307.576,281.317,226.233
Error,39.447327,50.719254,19.374283,17.777344,37.301559,15.31337,7.447784,7.825546,23.470734,19.881958,27.669556,60.130783,36.675919,44.083267,10.121902,21.378616,32.005539,33.626373,7.689621,0.228867,23.330902,49.641922,114.962753,155.314224,282.747803,337.05191,323.417236,345.672638,318.833923,221.285248


In [32]:
display(mae)
display(mape)

89.48094

0.32476747

In [33]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [34]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             train_split)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 13ms/step
Ano-0: |Prediction[[2716.8787]] - Target[2942.3769999999995]| =  Error: [[225.49829]]; MAPE:[[0.07663814]]
1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[2635.9478]] - Target[3097.755]| =  Error: [[461.80713]]; MAPE:[[0.14907801]]
1/1 [==============================] - 0s 14ms/step
Ano-5: |Prediction[[-126.52276]] - Target[1702.4859999999999]| =  Error: [[1829.0087]]; MAPE:[[1.0743165]]


[array([[225.49829]], dtype=float32),
 array([[461.80713]], dtype=float32),
 array([[1829.0087]], dtype=float32)]

838.7713

0.43334422